In [1]:
!pip install openai==0.28
!pip install jsonschema

In [ ]:
import json
import openai
import threading
import re
from jsonschema import validate, ValidationError

# Directly set the API key
openai.api_key = 'sk-proj-IuNe5dmEfg3JjN0vUa7YT3BlbkFJ9li48prvLGsegUVTRyba'

# Define a dictionary with weather data (temperature for each location)
weather_data_dict = {
    'new york': 25,
    'mumbai': 34,
    'jaipur': 21,
    'manali': 5
}

# Define the JSON schema
weather_schema = {
  "name": "get_weather",
  "description": "Determine weather in my location",
  "parameters": {
    "type": "object",
    "properties": {
      "location": {
        "type": "string",
        "description": "The city and state e.g. San Francisco, CA"
      },
      "unit": {
        "type": "string",
        "enum": [
          "c",
          "f"
        ]
      }
    },
    "required": [
      "location"
    ]
  }
}

def get_weather(location, unit='c'):
    # Look up the weather data based on the location (case-neutral)
    location = location.lower()
    temperature = weather_data_dict.get(location)
    if temperature is None:
        return json.dumps({
            "location": location,
            "temperature": "N/A",
            "description": "Data not available"
        }, indent=4)
    
    # Convert temperature to Fahrenheit if needed
    if unit == 'f':
        temperature = (temperature * 9/5) + 32
    
    # Set the description based on temperature criteria
    if temperature < 20:
        description = "cloudy sky"
    elif 20 <= temperature <= 30:
        description = "clear sky"
    else:
        description = "sunny sky"
    
    weather_data = {
        "location": location,
        "temperature": f"{temperature}°{'F' if unit == 'f' else 'C'}",
        "description": description
    }
    return json.dumps(weather_data, indent=4)

def get_assistant_response(messages):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=150
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"Error: {str(e)}"

def extract_location(user_input):
    # This is a simple regex to extract a location after the word 'in'
    match = re.search(r"in\s([a-zA-Z\s]+)", user_input)
    if match:
        return match.group(1).strip()
    return None

def handle_user_request(user_input):
    if "weather" in user_input.lower():
        location = extract_location(user_input)
        if location:
            try:
                weather_request = {
                    "location": location,
                    "unit": "c"
                }
                validate(instance=weather_request, schema=weather_schema["parameters"])
                weather_json = get_weather(location)
                weather_data = json.loads(weather_json)
                return (f"The current temperature in {weather_data['location']} is {weather_data['temperature']} "
                        f"with {weather_data['description']}.")
            except ValidationError as e:
                return f"Invalid input: {e.message}"
        else:
            return "Could not determine location. Please provide the location in your request."
    else:
        # Create a conversation history for the assistant
        messages = [
            {"role": "system", "content": "You are a knowledgeable assistant capable of answering a wide range of questions."},
            {"role": "user", "content": user_input}
        ]
        return get_assistant_response(messages)

def handle_input():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        
        response = handle_user_request(user_input)
        print("Assistant:", response)

def threaded_input():
    input_thread = threading.Thread(target=handle_input)
    input_thread.start()
    input_thread.join()  # Wait for the thread to finish

# Start the threaded input
threaded_input()


You: weather in new york
Assistant: The current temperature in new york is 25°C with clear sky.
You: capital of india
Assistant: The capital of India is New Delhi.
You: year of indian independence
Assistant: India gained independence from British colonial rule on August 15, 1947.
You: weather in mumbai
Assistant: The current temperature in mumbai is 34°C with sunny sky.
You: the greatest scientist of our time.
Assistant: It is subjective to determine the greatest scientist of our time as there have been many influential scientists across different fields. However, some of the most widely recognized scientists in recent times include Stephen Hawking for his contributions to theoretical physics and cosmology, Jane Goodall for her groundbreaking work in primatology and conservation, and Jennifer Doudna and Emmanuelle Charpentier for their development of CRISPR gene editing technology. These are just a few examples, and the field of science is vast with many brilliant minds making signific